## 01 - notebook for simple feature finding based on opencv-python

#### Contributor(s): Utkarsh Pratiush <utkarshp1161@gmail.com> - 10th June 2025
#### edited - 
   

In [ ]:
from stemOrchestrator.logging_config   import setup_logging
data_folder  = "."
out_path = data_folder
setup_logging(out_path=out_path) 

In [ ]:
from stemOrchestrator.acquisition import TFacquisition, DMacquisition
from stemOrchestrator.simulation import DMtwin
from stemOrchestrator.process import HAADF_tiff_to_png, tiff_to_png
from autoscript_tem_microscope_client import TemMicroscopeClient
import matplotlib.pyplot as plt
import logging
from typing import Dict
import os

In [ ]:
import os
import json
from pathlib import Path

ip = os.getenv("MICROSCOPE_IP")
port = os.getenv("MICROSCOPE_PORT")

if not ip or not port:
    secret_path = Path("../../config_secret.json")
    if secret_path.exists():
        with open(secret_path, "r") as f:
            secret = json.load(f)
            ip = ip or secret.get("ip_TF_wd")
            port = port or secret.get("port_TF_wd")



config = {
    "ip": ip,
    "port": port,
    "haadf_exposure": 40e-8,  # micro-seconds per pixel
    "haadf_resolution": 512, # square
    "out_path": "."
}

In [ ]:

# ip = config["ip"]
port = config["port"]
haadf_exposure = config["haadf_exposure"]
out_path = config["out_path"]
haadf_resolution = config["haadf_resolution"]



microscope = TemMicroscopeClient()
microscope.connect(ip, port = port)# 7521 on velox  computer

# query state:

tf_acquisition = TFacquisition(microscope=microscope)

# put beam shift to 0,0
# tf_acquisition.move_beam_shift_positon([0, 0])


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.measure import label
from skimage.color import label2rgb

# Acquire HAADF image
haadf_np_array, haadf_tiff_name = tf_acquisition.acquire_haadf(exposure=10e-6, resolution=512)
HAADF_tiff_to_png(haadf_tiff_name)

# Normalize image
img = haadf_np_array.astype(np.float32)
img = (img - img.min()) / (img.max() - img.min())  # 0-1 normalization
img_uint8 = (img * 255).astype(np.uint8)

# Smooth to reduce noise (optional)
blurred = cv2.GaussianBlur(img_uint8, (3, 3), 0)

# Detect edges and make binary mask
edges = cv2.Canny(blurred, threshold1=50, threshold2=150)

# Invert and fill particles using connected components
inverted = cv2.bitwise_not(edges)
_, binary = cv2.threshold(inverted, 128, 255, cv2.THRESH_BINARY)

# Label connected regions (particles)
labels = label(binary)

# Color each region
colored_particles = label2rgb(labels, image=img, bg_label=0)

# Plot
plt.figure(figsize=(12, 5))
plt.subplot(1, 3, 1)
plt.title("Original HAADF")
plt.imshow(img_uint8, cmap='gray')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.title("Edge Mask")
plt.imshow(edges, cmap='gray')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.title("Colored Particles")
plt.imshow(colored_particles)
plt.axis('off')

plt.tight_layout()
plt.show()
